In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys
import torch
# Does this fix the hdf5 multiprocessing bug?
torch.multiprocessing.set_start_method("spawn")

# !git clone https://github.com/guydav/deep-learning-projects.git
# !pip install -q -r deep-learning-projects/requirements.txt
!wandb login 9676e3cc95066e4865586082971f2653245f09b4
sys.path.extend(('/home/cc/deep-learning-projects', '/home/cc/src/tqdm'))

In [ ]:
import projects
from projects.metalearning import *

## Load the dataset

This time with 3x the batch size, 1.5k

In [ ]:
ML_50K = '/home/cc/meta_learning_50k.h5'

benchmark_dimension = 0
dataset_random_seed = 33
train_coreset_size = 3000
test_coreset_size = 5000

np.random.seed(0)
query_order = np.random.shuffle(np.arange(10))
accuracy_threshold = 0.9

normalized_train_dataset, train_dataloader, normalized_test_dataset, test_dataloader = \
        create_normalized_datasets(dataset_path=ML_50K, 
                                   batch_size=1500,
                                   num_workers=4,
                                   downsample_size=None,
                                   should_flip=False,
                                   return_indices=False,
                                   dataset_class=SequentialBenchmarkMetaLearningDataset,
                                   dataset_class_kwargs=dict(
                                       benchmark_dimension=benchmark_dimension,
                                       random_seed=dataset_random_seed,
                                       query_order=query_order
                                   ),
                                   train_dataset_kwargs=dict(previous_query_coreset_size=train_coreset_size),
                                   test_dataset_kwargs=dict(previous_query_coreset_size=test_coreset_size))

In [ ]:
current_epoch = 0
save_dir = '/home/cc/checkpoints'

sequential_benchmark_test_model = PoolingDropoutCNNMLP(
    query_length=30,
    conv_filter_sizes=(16, 32, 48, 64),
    conv_output_size=4480, 
    mlp_layer_sizes=(512, 512, 512, 512),
    lr=5e-4, 
    weight_decay=1e-4, 
    conv_dropout=False,
    mlp_dropout=False,
    name='no_dropout_yes_decay_sequential_benchmark_v1',
    save_dir=save_dir)

sequential_benchmark_test_model.load_model(current_epoch)
sequential_benchmark_test_model = sequential_benchmark_test_model.cuda()

In [ ]:
# os.environ['WANDB_RUN_ID'] ='98w3kzlw'
# os.environ['WANDB_RESUME'] = 'must'
wandb.init(entity='meta-learning-scaling', project='sequential-benchmark') #, resume=True)

current_model = sequential_benchmark_test_model

wandb.config.lr = current_model.lr
wandb.config.decay = current_model.weight_decay
wandb.config.loss = 'CE'
wandb.config.batch_size = train_dataloader.batch_size
wandb.config.benchmark_dimension = benchmark_dimension
wandb.config.dataset_random_seed = dataset_random_seed
wandb.config.train_coreset_size = train_coreset_size
wandb.config.test_coreset_size = test_coreset_size
wandb.config.query_order = query_order
wandb.config.accuracy_threshold = accuracy_threshold

In [ ]:
total_epochs = 200
wandb.config.update(dict(epochs=total_epochs), allow_val_change=True)

sequential_benchmark(sequential_benchmark_test_model, train_dataloder, test_dataloader, accuracy_threshold,
                     num_epochs=total_epochs - current_epoch, epochs_to_graph=10, start_epoch=current_epoch)